<a href="https://colab.research.google.com/github/isadays/NLP/blob/main/AccesingGPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np
import time
import os

# Suppress TensorFlow logging for clarity
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# Check TensorFlow version and GPU availability
print("TensorFlow Version:", tf.__version__)
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print("GPUs Available:")
    for gpu in gpus:
        print(f"  - {gpu.name}")
else:
    print("No GPUs available.")



TensorFlow Version: 2.17.1
GPUs Available:
  - /physical_device:GPU:0


In [2]:
import torch
print("PyTorch Version:", torch.__version__)
print("Is CUDA Available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("CUDA Device Name:", torch.cuda.get_device_name(0))


PyTorch Version: 2.5.1+cu121
Is CUDA Available: True
CUDA Device Name: Tesla T4


In [ ]:
# Load and preprocess the MNIST dataset
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

def train_model(device, x_train, y_train, x_test, y_test, epochs=5, batch_size=64):
    with tf.device(device):
        # Build the model
        model = models.Sequential([
            layers.Flatten(input_shape=(28, 28)),
            layers.Dense(512, activation='relu'),
            layers.Dropout(0.2),
            layers.Dense(10, activation='softmax')
        ])

        # Compile the model
        model.compile(optimizer='adam',
                      loss='sparse_categorical_crossentropy',
                      metrics=['accuracy'])

        # Train the model
        start_time = time.time()
        history = model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=0.1, verbose=0)
        end_time = time.time()

        training_time = end_time - start_time

        # Evaluate the model
        test_loss, test_acc = model.evaluate(x_test, y_test, verbose=0)

        return training_time, test_acc




In [ ]:
# Train on CPU
print("\nTraining on CPU...")
cpu_time, cpu_acc = train_model('/CPU:0', x_train, y_train, x_test, y_test)
print(f"CPU Training Time: {cpu_time:.2f} seconds")
print(f"CPU Test Accuracy: {cpu_acc:.4f}")

# Train on GPU
print("\nTraining on GPU...")
gpu_time, gpu_acc = train_model('/GPU:0', x_train, y_train, x_test, y_test)
print(f"GPU Training Time: {gpu_time:.2f} seconds")
print(f"GPU Test Accuracy: {gpu_acc:.4f}")

# Compare Results
print("\n=== Training Time Comparison ===")
print(f"CPU Training Time: {cpu_time:.2f} seconds")
print(f"GPU Training Time: {gpu_time:.2f} seconds")

print("\n=== Test Accuracy Comparison ===")
print(f"CPU Test Accuracy: {cpu_acc:.4f}")
print(f"GPU Test Accuracy: {gpu_acc:.4f}")

In [3]:
# =========================================
# TensorFlow and PyTorch GPU vs. CPU Comparison
# =========================================

# Import Necessary Libraries
import tensorflow as tf
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from tensorflow.keras import layers, models
import numpy as np
import time
import os

# =========================================
# Section 1: Setup and Environment Verification
# =========================================

# Suppress TensorFlow and PyTorch Logging for Clarity
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # TensorFlow: 0 = all logs, 1 = filter out INFO, 2 = filter out WARNING, 3 = filter out ERROR
os.environ['TORCH_CPP_LOG_LEVEL'] = '2'   # PyTorch: 0 = all logs, 1 = WARNING, 2 = ERROR

print("========== Environment Verification ==========\n")

# TensorFlow Version and GPU Availability
print("TensorFlow Version:", tf.__version__)
tf_gpus = tf.config.list_physical_devices('GPU')
if tf_gpus:
    print("TensorFlow GPUs Available:")
    for gpu in tf_gpus:
        print(f"  - {gpu.name}")
else:
    print("No TensorFlow GPUs available.")

# PyTorch Version and CUDA Availability
print("\nPyTorch Version:", torch.__version__)
print("Is CUDA Available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("CUDA Device Name:", torch.cuda.get_device_name(0))
else:
    print("No CUDA-enabled GPU available for PyTorch.")

print("\n==============================================\n")



========== Environment Verification ==========

TensorFlow Version: 2.17.1
TensorFlow GPUs Available:
  - /physical_device:GPU:0

PyTorch Version: 2.5.1+cu121
Is CUDA Available: True
CUDA Device Name: Tesla T4




In [4]:
# =========================================
# Section 2: Data Loading and Preprocessing
# =========================================

print("========== Data Loading and Preprocessing ==========\n")

# Load and Preprocess the MNIST Dataset for TensorFlow
mnist = tf.keras.datasets.mnist
(x_train_tf, y_train_tf), (x_test_tf, y_test_tf) = mnist.load_data()
x_train_tf, x_test_tf = x_train_tf / 255.0, x_test_tf / 255.0

print("TensorFlow MNIST data loaded and normalized.")

# Load and Preprocess the MNIST Dataset for PyTorch
# Convert data to PyTorch tensors and add channel dimension
x_train_pt = torch.tensor(x_train_tf, dtype=torch.float32).unsqueeze(1)  # Shape: (60000, 1, 28, 28)
x_test_pt = torch.tensor(x_test_tf, dtype=torch.float32).unsqueeze(1)    # Shape: (10000, 1, 28, 28)
y_train_pt = torch.tensor(y_train_tf, dtype=torch.long)
y_test_pt = torch.tensor(y_test_tf, dtype=torch.long)

# Create TensorDatasets and DataLoaders for PyTorch
train_dataset_pt = TensorDataset(x_train_pt, y_train_pt)
test_dataset_pt = TensorDataset(x_test_pt, y_test_pt)

train_loader_pt = DataLoader(train_dataset_pt, batch_size=64, shuffle=True)
test_loader_pt = DataLoader(test_dataset_pt, batch_size=64, shuffle=False)

print("PyTorch MNIST data loaded, converted to tensors, and DataLoaders created.\n")


========== Data Loading and Preprocessing ==========

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
TensorFlow MNIST data loaded and normalized.
PyTorch MNIST data loaded, converted to tensors, and DataLoaders created.



In [5]:
# =========================================
# Section 3: Define Training and Evaluation Functions
# =========================================

print("========== Defining Training and Evaluation Functions ==========\n")

# ---------------------
# TensorFlow Training Function
# ---------------------
def train_tensorflow_model(device, x_train, y_train, x_test, y_test, epochs=5, batch_size=64):
    with tf.device(device):
        # Build the TensorFlow model
        model = models.Sequential([
            layers.Flatten(input_shape=(28, 28)),
            layers.Dense(512, activation='relu'),
            layers.Dropout(0.2),
            layers.Dense(10, activation='softmax')
        ])

        # Compile the model
        model.compile(optimizer='adam',
                      loss='sparse_categorical_crossentropy',
                      metrics=['accuracy'])

        # Train the model
        start_time = time.time()
        history = model.fit(x_train, y_train,
                            epochs=epochs,
                            batch_size=batch_size,
                            validation_split=0.1,
                            verbose=0)  # Set verbose=1 to see training progress
        end_time = time.time()

        training_time = end_time - start_time

        # Evaluate the model
        test_loss, test_acc = model.evaluate(x_test, y_test, verbose=0)

        return training_time, test_acc

# ---------------------
# PyTorch Training Function
# ---------------------
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(28 * 28, 512)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.2)
        self.fc2 = nn.Linear(512, 10)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        x = self.softmax(x)
        return x

def train_pytorch_model(device, train_loader, model, criterion, optimizer, epochs=5):
    model.to(device)
    model.train()
    start_time = time.time()
    for epoch in range(epochs):
        for data, target in train_loader:
            data, target = data.to(device), target.to(device)

            # Zero the parameter gradients
            optimizer.zero_grad()

            # Forward pass
            outputs = model(data)
            loss = criterion(outputs, target)

            # Backward pass and optimization
            loss.backward()
            optimizer.step()
    end_time = time.time()
    training_time = end_time - start_time
    return training_time

def evaluate_pytorch_model(device, test_loader, model):
    model.to(device)
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            outputs = model(data)
            _, predicted = torch.max(outputs.data, 1)
            total += target.size(0)
            correct += (predicted == target).sum().item()
    accuracy = correct / total
    return accuracy

print("Training and evaluation functions defined.\n")



========== Defining Training and Evaluation Functions ==========

Training and evaluation functions defined.



In [6]:
# =========================================
# Section 4: TensorFlow Training on CPU and GPU
# =========================================

print("========== TensorFlow Training ==========\n")

# Train on CPU
print("--- TensorFlow Training on CPU ---")
cpu_time_tf, cpu_acc_tf = train_tensorflow_model('/CPU:0', x_train_tf, y_train_tf, x_test_tf, y_test_tf)
print(f"CPU Training Time: {cpu_time_tf:.2f} seconds")
print(f"CPU Test Accuracy: {cpu_acc_tf:.4f}\n")

# Train on GPU
if tf_gpus:
    print("--- TensorFlow Training on GPU ---")
    gpu_time_tf, gpu_acc_tf = train_tensorflow_model('/GPU:0', x_train_tf, y_train_tf, x_test_tf, y_test_tf)
    print(f"GPU Training Time: {gpu_time_tf:.2f} seconds")
    print(f"GPU Test Accuracy: {gpu_acc_tf:.4f}\n")
else:
    print("GPU training not available for TensorFlow.\n")

# =========================================
# Section 5: PyTorch Training on CPU and GPU
# =========================================

print("========== PyTorch Training ==========\n")

# Initialize PyTorch Model, Loss Function, and Optimizer for CPU
model_cpu_pt = SimpleNN()
criterion_cpu_pt = nn.CrossEntropyLoss()
optimizer_cpu_pt = optim.Adam(model_cpu_pt.parameters(), lr=0.001)

# Train on CPU
print("--- PyTorch Training on CPU ---")
cpu_time_pt, cpu_acc_pt = 0, 0  # Initialize
cpu_time_pt = train_pytorch_model('cpu', train_loader_pt, model_cpu_pt, criterion_cpu_pt, optimizer_cpu_pt, epochs=5)
cpu_acc_pt = evaluate_pytorch_model('cpu', test_loader_pt, model_cpu_pt)
print(f"CPU Training Time: {cpu_time_pt:.2f} seconds")
print(f"CPU Test Accuracy: {cpu_acc_pt:.4f}\n")

# Train on GPU
if torch.cuda.is_available():
    # Initialize PyTorch Model, Loss Function, and Optimizer for GPU
    model_gpu_pt = SimpleNN()
    criterion_gpu_pt = nn.CrossEntropyLoss()
    optimizer_gpu_pt = optim.Adam(model_gpu_pt.parameters(), lr=0.001)

    print("--- PyTorch Training on GPU ---")
    gpu_time_pt, gpu_acc_pt = 0, 0  # Initialize
    gpu_time_pt = train_pytorch_model('cuda', train_loader_pt, model_gpu_pt, criterion_gpu_pt, optimizer_gpu_pt, epochs=5)
    gpu_acc_pt = evaluate_pytorch_model('cuda', test_loader_pt, model_gpu_pt)
    print(f"GPU Training Time: {gpu_time_pt:.2f} seconds")
    print(f"GPU Test Accuracy: {gpu_acc_pt:.4f}\n")
else:
    print("GPU training not available for PyTorch.\n")


========== TensorFlow Training ==========

--- TensorFlow Training on CPU ---


/usr/local/lib/python3.11/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


CPU Training Time: 24.08 seconds
CPU Test Accuracy: 0.9806

--- TensorFlow Training on GPU ---
GPU Training Time: 12.22 seconds
GPU Test Accuracy: 0.9781

========== PyTorch Training ==========

--- PyTorch Training on CPU ---
CPU Training Time: 26.49 seconds
CPU Test Accuracy: 0.9701

--- PyTorch Training on GPU ---
GPU Training Time: 9.99 seconds
GPU Test Accuracy: 0.9743



In [7]:

# =========================================
# Section 6: Results Comparison
# =========================================

print("========== Results Comparison ==========\n")

# TensorFlow Results
print("=== TensorFlow Training Time Comparison ===")
print(f"CPU Training Time: {cpu_time_tf:.2f} seconds")
if tf_gpus:
    print(f"GPU Training Time: {gpu_time_tf:.2f} seconds")
print()

print("=== TensorFlow Test Accuracy Comparison ===")
print(f"CPU Test Accuracy: {cpu_acc_tf:.4f}")
if tf_gpus:
    print(f"GPU Test Accuracy: {gpu_acc_tf:.4f}")
print("\n----------------------------------------------\n")

# PyTorch Results
print("=== PyTorch Training Time Comparison ===")
print(f"CPU Training Time: {cpu_time_pt:.2f} seconds")
if torch.cuda.is_available():
    print(f"GPU Training Time: {gpu_time_pt:.2f} seconds")
print()

print("=== PyTorch Test Accuracy Comparison ===")
print(f"CPU Test Accuracy: {cpu_acc_pt:.4f}")
if torch.cuda.is_available():
    print(f"GPU Test Accuracy: {gpu_acc_pt:.4f}")
print("\n==============================================\n")



========== Results Comparison ==========

=== TensorFlow Training Time Comparison ===
CPU Training Time: 24.08 seconds
GPU Training Time: 12.22 seconds

=== TensorFlow Test Accuracy Comparison ===
CPU Test Accuracy: 0.9806
GPU Test Accuracy: 0.9781

----------------------------------------------

=== PyTorch Training Time Comparison ===
CPU Training Time: 26.49 seconds
GPU Training Time: 9.99 seconds

=== PyTorch Test Accuracy Comparison ===
CPU Test Accuracy: 0.9701
GPU Test Accuracy: 0.9743


